In [1]:
!pip install -qq langchain wget cohere
!pip -q install streamlit

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 783.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.2/249.2 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 111.9 MB/s eta 0:00:00
   ━━━━━

In [2]:
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-llama-cpp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 861.9/861.9 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 MB 7.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.4 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=ll

In [19]:
!set LLAMA_CUBLAS=1
!set CMAKE_ARGS=-DLLAMA_CUBLAS=on
!set FORCE_CMAKE=1
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

In [3]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/Projects/ChatBot/")

Mounted at /content/drive


In [4]:
# Custom progress bar function
import wget
def bar_custom(current, total, width=80):
    progress_message = f"Downloading: {current / total * 100:.2f}% [{current}/{total}] bytes"
    print(progress_message, end="\r")

# URL of the file to download
model_url = "https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q2_K.gguf"

# Download the file with custom progress bar
filename = wget.download(model_url, bar=bar_custom)

print(f"\nDownload completed. Saved as: {filename}")


Download completed. Saved as: llama-2-7b-chat.Q2_K.gguf


In [3]:
!ls

app.py	       llama-2-7b-chat.Q2_K.gguf  node_modules	package-lock.json
chatbot.ipynb  logo.jpg			  package.json


In [20]:
%%writefile app.py
import streamlit as st
from langchain.schema import(SystemMessage, HumanMessage, AIMessage)
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)

## Initalise the page using streamlit app
def init_page():
  st.set_page_config(page_title="ChatBot", page_icon="🤖")
  st.sidebar.image('logo.jpg', width=100)
  st.sidebar.title("AI ChatBot")
  st.sidebar.subheader("Intelligent bot to answer all your questions at the tap of your finger")

def init_messages():
  clear_button = st.sidebar.button("Clear Conversation", key="clear")
  if clear_button or "messages" not in st.session_state:
    #Setting the system message to make sure that BOT understand its role and behavior
    st.session_state.messages = [
      SystemMessage(
        content="You are intelligent AI assistant. Reply your answer in markdown format."
      )
    ]

def initialize_model():
  return LlamaCPP(
    model_path="llama-2-7b-chat.Q2_K.gguf",
    temperature=0.7,
    max_new_tokens=500,
    context_window=3900,
    generate_kwargs={},
    model_kwargs={"n_gpu_layers":64},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
  )

def get_answer(llm, prompt):
    response = llm.complete(prompt)
    response_text = response.text
    return response_text


def main():
  init_page()
  init_messages()
  llm = initialize_model()

  if user_input := st.chat_input("Input your question!"):
    st.session_state.messages.append(HumanMessage(content=user_input))
    with st.spinner("ChatBot is typing ..."):
      answer = get_answer(llm, user_input)
    st.session_state.messages.append(AIMessage(content=answer))


  messages = st.session_state.get("messages", [])
  for message in messages:
    if isinstance(message, AIMessage):
      with st.chat_message("AI Assistant"):
        st.markdown(message.content)
    elif isinstance(message, HumanMessage):
      with st.chat_message("Human"):
        st.markdown(message.content)

if __name__ == "__main__":
  main()

Overwriting app.py


In [9]:
import torch
torch.cuda.is_available()

True

In [21]:
## Get External IP|
!wget -q -O - ipv4.icanhazip.com

35.185.179.192


In [6]:
## Install localtunnel
!npm install -g localtunnel


added 22 packages, and audited 23 packages in 4s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [22]:
!streamlit run /content/drive/MyDrive/Projects/ChatBot/app.py &>/content/logs.txt &
!npx localtunnel --port 8501

your url is: https://plain-eggs-spend.loca.lt
^C
